In [1]:
import sys
from pytube import YouTube
from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import crop
from os import path, remove, makedirs, rename,system, name as operating_system_name
import os
from cv2 import VideoCapture,cvtColor, absdiff,COLOR_BGR2RGB,COLOR_BGR2GRAY,CAP_PROP_FPS,CAP_PROP_FRAME_COUNT
from mediapipe.python.solutions.face_mesh import FaceMesh
from mediapipe.python.solutions.face_detection import FaceDetection
from pandas import read_csv, concat,DataFrame
from numpy import mean
from time import sleep
import pysrt
import re 
from string import punctuation
from subprocess import run, Popen
from glob import glob
from shutil import copy, move
from matplotlib.pyplot import plot 
from gc import collect
import threading
import pickle
import datetime
import time
from subprocess import run, Popen,DEVNULL, PIPE, STDOUT


pygame 2.5.2 (SDL 2.28.3, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import sys
sys.executable.replace("\\","/")

'd:/Code-Projects/University/Fall2023/Grad/LipNet-Assistive-AI/.conda/python.exe'

In [3]:
WORK_DIR = r"E:/Video Links Dataset"
CLIPPED_VIDEOS = WORK_DIR + r"/clipped/videos"
CLIPPED_ALIGNMENTS = WORK_DIR + r"/clipped/alignments"
DOWNLOADED_VIDEOS = WORK_DIR + r"/downloaded/videos"
DOWNLOADED_ALIGNMENTS = WORK_DIR + r"/downloaded/alignments"
FILTERED_VIDEOS = WORK_DIR + r"/filtered/videos"
FILTERED_ALIGNMENTS = WORK_DIR + r"/filtered/alignments"
LINK_DATASET = WORK_DIR + r"/Video Links Dataset.csv"
STATUS_DATASET = WORK_DIR + r"/Video Status.csv"
LOG_FILE_DOWN = WORK_DIR + r"/log_down.txt"
LOG_FILE_CLIP = WORK_DIR + r"/log_clip.txt"
LOG_FILE_FILT = WORK_DIR + r"/log_filt.txt"
PICKELS = WORK_DIR + r"/pickles"
DOWNLOADER = r"scripts/download_video.py"
CLIPPER = r"scripts/clip_video.py"
FILTERER = r'scripts/filter_videos.py'
RUNTIME = sys.executable.replace('\\','/')


In [4]:
### logging and control functions ###
def empty_dir(dir):
    for i in glob(dir+r"/*"):
        remove(i)
        
def log(text,log_file):
    try:
        with open(log_file, 'a') as file:
            file.write(text)
    except Exception as e:
        print("Failed to log: ",text, e)
        
def save_pipeline_status(video_idx,status):
    video_idx_1 =  int(video_idx) 
    df = read_csv(LINK_DATASET)
    df["status"][video_idx_1] = status
    df.to_csv(LINK_DATASET,index=False)    

def log_status_video(video_id_obj):
    status_df = read_csv(STATUS_DATASET)
    status_df = concat([status_df, DataFrame([video_id_obj])], ignore_index=True)
    status_df.to_csv(STATUS_DATASET,index=False)    


In [5]:
def clear_console():
    system('cls' if operating_system_name=='nt' else 'clear')
    
def serialize_object(obj, name):

    with open(name + '.pickle', 'wb') as f:
        pickle.dump(obj, f)

def check_outliers(numbers,threshold):

    mean = sum(numbers) / len(numbers)
    
    # Calculate standard deviation
    variance = sum((x - mean) ** 2 for x in numbers) / len(numbers)
    std_dev = variance ** 0.5
    
    # Define thresholds
    upper_threshold = mean + (threshold * std_dev)
    
    # Check if any number falls outside the thresholds
    for num in numbers:
        if num > upper_threshold:
            return True,upper_threshold  # Return True if any number is outside the thresholds
    
    return False,upper_threshold  # Return False if no number is outside the thresholds

def calculate_frame_differences(video_path):
    cap = VideoCapture(video_path)
    if not cap.isOpened():
        # print("Error: Couldn't open video file.")
        return None
    
    # Read the first frame
    ret, prev_frame = cap.read()
    if not ret:
        # print("Error: Couldn't read the first frame.")
        return None
    
    # Initialize list to store differences
    differences = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Calculate absolute difference between frames
        diff = absdiff(prev_frame, frame)
        
        # Convert difference image to grayscale
        gray_diff = cvtColor(diff, COLOR_BGR2GRAY)
        
        # Calculate the average pixel difference
        avg_diff = mean(gray_diff)
        differences.append(avg_diff)
        
        # Update the previous frame
        prev_frame = frame.copy()
    cap.release()
    # plt.plot(differences);
    return check_outliers(differences,4)

def english_to_arabic_numbers(text):

    numbers_dict = {
        '0': '٠',
        '1': '١',
        '2': '٢',
        '3': '٣',
        '4': '٤',
        '5': '٥',
        '6': '٦',
        '7': '٧',
        '8': '٨',
        '9': '٩',
    }
    for eng_num, arabic_num in numbers_dict.items():
        text = text.replace(eng_num, arabic_num)
    return text

def remove_special_char(text):
    for i in punctuation:
        text = text.replace(i, " ")

    return text

def remove_repeating_spaces(text):
    cleaned_text = re.sub(r' +', ' ', text)
    return cleaned_text

def check_if_not_arabic(text):
    ar = "ابتثجحخدذرزسشصضطظعغفقكلمنهويىء٠١٢٣٤٥٦٧٨٩ "
    for i in text:
        if i not in ar:
            return True
    return False

def get_video(url, video_id):
    try:
        yt = YouTube(url)

        stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()

        if "a.ar" in yt.captions.keys():
            captions = yt.captions.get_by_language_code("a.ar")
            captions.download(
                    srt=True,
                    output_path=DOWNLOADED_ALIGNMENTS,
                    title=f"{video_id}")

            stream.download(
                filename=fr"{DOWNLOADED_VIDEOS}/{video_id}.mp4"
            )

            return fr"{DOWNLOADED_VIDEOS}/{video_id}.mp4",fr"{DOWNLOADED_ALIGNMENTS}/{video_id} (a.ar).srt"

        else:
            return False, False
        
    except Exception as e:
        return None,None

def video_is_accpeted(video_id):

    video_path = CLIPPED_VIDEOS+fr"/T_{video_id}.mp4"
 
    video_status_obj =  {
        "video_id":video_id,	
        "duration":None,
        "fps":None,
        "width":None,	
        "height":None,	
        "no_face_frames":0,	
        "max_num_faces":0,	
        "no_lips_frames":0,
        "correct_orientation_frames":0,
        "upper_lip_threshold":None,
        "upper_frame_diff_threshold":None,
	    "min_lip_diff":None,
        "max_lip_diff":None,
        "median_lip_diff":None,
        "mean_lip_diff":None,
        "min_video_diff":None,
        "max_video_diff":None,
        "median_video_diff":None,
        "mean_video_diff":None}

    try:
        # open video file
        cap = VideoCapture(video_path)
        if not cap.isOpened():
            log_status_video(video_status_obj)
            cap.release()
            return
            # raise Exception("can't open video file")
        
        video_fps = int(cap.get(CAP_PROP_FPS))
        video_frames = int(cap.get(CAP_PROP_FRAME_COUNT))
        video_duration = video_frames // video_fps

        # if (int(video_duration) < 2) or (int(video_duration) > 5):
        #     cap.release()
        #     raise Exception("video_duration must be between 2 and 5")
        
        # if((video_fps < 25) or (video_fps > 60)):
        #     cap.release()
        #     raise Exception("video fps must be between 25 and 60")
        video_status_obj['fps'] = video_fps
        video_status_obj['duration'] = video_duration

        # lip coordinates,
        lip_pairs = [[13, 14], [82, 87], [312, 317],[81,178],[311,402],[310,318],[80,88]]
        # list for whether face is facing us
        orientation_list=[]
        # list for whether frames are lips are moving rapidly
        lip_diffs = []

        # loop over all frames in the video
        for _ in range(int(cap.get(CAP_PROP_FRAME_COUNT))):
            ret, frame = cap.read()
            if not ret:
                # cap.release()
                continue
                # raise Exception("couldn't read frame")
          
          
            if video_status_obj.get("width") == None:
                video_status_obj['width'] = frame.shape[0]
                video_status_obj['height'] = frame.shape[1]
                
            
            # convert to rgb
            frame = cvtColor(frame, COLOR_BGR2RGB)

            # get the keypoints (x,y)s of each point of a face in the frame
            results = face_detection.process(frame)

            if not results.detections:
                # cap.release()
                # raise Exception("no faces detected")
                video_status_obj['no_face_frames'] += 1

            video_status_obj['max_num_faces'] = max(video_status_obj['max_num_faces'],len(results.detections))

            # cap.release()
            # raise Exception("wrong number of faces detected")

            face_mesh_result = face_mesh.process(frame)

            if not face_mesh_result.multi_face_landmarks:
                # cap.release()
                # raise Exception("no lips detected")
                video_status_obj['no_lips_frames'] += 1
            
            if (len(results.detections) !=1) or (not face_mesh_result.multi_face_landmarks) :
                continue
                
            for face_landmarks in face_mesh_result.multi_face_landmarks:

                landmarks = face_landmarks.landmark
                right_x = int(landmarks[454].x * frame.shape[1])
                left_x = int(landmarks[234].x * frame.shape[1])
                upper_x = int(landmarks[10].x * frame.shape[1])
                lower_x = int(landmarks[152].x * frame.shape[1])

                center_x = (upper_x + lower_x)//2

                if(left_x>center_x):
                    orientation_list.append(0)
                elif(right_x<center_x):
                    orientation_list.append(0)
                else:
                    orientation_list.append(1)
                    
                diff = 0
                for lip_pair in lip_pairs:
                    upper_lip_x, upper_lip_y = int(
                        landmarks[lip_pair[0]].x * frame.shape[1]
                    ), int(landmarks[lip_pair[0]].y * frame.shape[0])
                    lower_lip_x, lower_lip_y = int(
                        landmarks[lip_pair[1]].x * frame.shape[1]
                    ), int(landmarks[lip_pair[1]].y * frame.shape[0])
                    diff += ((upper_lip_x - lower_lip_x) ** 2) + (
                        (upper_lip_y - lower_lip_y) ** 2
                    )
                lip_diffs.append(diff)

        video_status_obj['upper_frame_diff_threshold'] = calculate_frame_differences(video_path)[1]
        
        if (len(orientation_list) != 0):
            video_status_obj['correct_orientation_frames'] = sum(orientation_list)/len(orientation_list)
            

        if (len(lip_diffs) != 0):
            # cap.release()
            # return 
            # raise Exception("Face border coudn't detect any face for orientation")
            video_status_obj['upper_lip_threshold'] = check_outliers(lip_diffs,1)[1]
            video_status_obj['min_lip_diff'] =  min(lip_diffs)

        if(len(lip_diffs) == 0 ):
            cap.release()
            raise Exception("Face mesh couldn't detect any lips")
        
        if((sum(orientation_list)/len(orientation_list)) !=1.0):
            cap.release()
            raise Exception("too much face angle")

        if(not check_outliers(lip_diffs,1)):
            cap.release()
            raise Exception("lips not moving")
        
        if calculate_frame_differences(video_path):
            cap.release()
            raise Exception("sudden change in frames")
        cap.release()

    except Exception as e:
        cap.release()
        raise e


In [6]:

# def clip_video(video_path, alignment_path,video_id):
#     log(f"Clipping videos: {video_path}\n\n")

#     def filter_srt_sub(text):
    
#         if(len(text) < 2):
#             return False

#         if(text == "[موسيقى]"):
#             return False
        
#         text = english_to_arabic_numbers(text)
#         text = remove_special_char(text)
#         text = remove_repeating_spaces(text)
        
#         if(check_if_not_arabic(text)):
#             return False
        
#         if(len(text) < 2):
#             return False
        
#         return text
#     try:
#         subs = pysrt.open(alignment_path)
#         video = VideoFileClip(video_path)

#         for idx, sub in enumerate(subs):
#             try:
#                 text = filter_srt_sub(sub.text.strip())

#                 if(not text):
#                     continue

#                 start_time = sub.start.ordinal /1000.0
#                 end_time = sub.end.ordinal / 1000.0     

#                 video_part = video.subclip(start_time, end_time)
#                 output_path = path.join(CLIPPED_VIDEOS, f"{video_id}_{idx+1}.mp4")
                
#                 with open(CLIPPED_ALIGNMENTS+f"/{video_id}_{idx+1}.align","w") as writer:
#                     writer.write(text.replace(" ","\n"))

#                 video_part.write_videofile(output_path,verbose=False, progress_bar=False) 
#             except:
#                 video.close()
#         video.close()
#         log(f"Finished Clipping videos: {video_path}\n\n")

#     except:
#         raise Exception("Failed to open srt file or video file")

# def filter_single_video(idx,sub,video_id,video):
#     def filter_srt_sub(text):
    
#         if(len(text) < 2):
#             return False

#         if(text == "[موسيقى]"):
#             return False
        
#         text = english_to_arabic_numbers(text)
#         text = remove_special_char(text)
#         text = remove_repeating_spaces(text)
        
#         if(check_if_not_arabic(text)):
#             return False
        
#         if(len(text) < 2):
#             return False
        
#         return text
    
#     output_video_path = path.join(CLIPPED_VIDEOS, f"{video_id}_{idx}.mp4").replace("\\","/")
#     output_alignment_path = CLIPPED_ALIGNMENTS+f"/{video_id}_{idx}.align"
    
#     try:
#         try:
#             text = filter_srt_sub(sub.text.strip())

#             if(not text):
#                 return

#             start_time = sub.start.ordinal /1000.0
#             end_time = sub.end.ordinal / 1000.0     

#             video_part = video.subclip(start_time, end_time)

#             with open(output_alignment_path,"w") as writer:
#                 writer.write(text.replace(" ","\n"))

#             video_part.write_videofile(output_video_path,verbose=False, progress_bar=False) 
#             threaded_videos.append(int(idx))

#         except:
#             video.close()
#             pass

#     except Exception as e:
#         video.close()
#         remove(output_alignment_path)
#         remove(output_video_path)
#         del thread_bank[idx]
#         return

#     video.close()
#     del thread_bank[idx]
#     return

# def filter_video_threading(video_path, alignment_path,video_id):
#     log(f"Clipping and filtering videos: {video_path}\n\n")
    
#     global thread_bank
#     thread_bank = {}
#     global threads
#     threads = []

#     try:
#         subs = pysrt.open(alignment_path)

#         for idx, sub in enumerate(subs):

#             start_time = time.time()

#             while True: 
#                 if(len(thread_bank.keys()) < MAX_THREADS):
#                     break
#                 if((time.time() - start_time >= 10)):
#                     for thread in threads:
#                         thread.join(timeout=1)
#                     threads = []
#                     thread_bank = {}

#             thread_bank[idx] = 0
#             thread = threading.Thread(target=filter_single_video, args=(idx,sub,video_id,VideoFileClip(video_path)),daemon=True)
#             thread.start()
#             threads.append(thread)

#         start_time = time.time()
#         closing = True
#         while closing:
#             if (time.time() - start_time >= 10) or (len(thread_bank) == 0):
#                 for thread in threads:
#                     thread.join(timeout=1)
#                 closing = False

#         log("\nFinished Clipping Videos\n\n")

#         for thread_video_idx in threaded_videos:
#             thread_video_idx = str(thread_video_idx)
#             try:
#                 output_video_path = path.join(CLIPPED_VIDEOS, f"{video_id}_{thread_video_idx}.mp4").replace("\\","/")
#                 output_alignment_path = CLIPPED_ALIGNMENTS+f"/{video_id}_{thread_video_idx}.align"
#                 video_is_accpeted(output_video_path)
#                 move(output_video_path,FILTERED_VIDEOS)
#                 move(output_alignment_path,FILTERED_ALIGNMENTS)
#                 log_status_video(f"{video_id}_{thread_video_idx}","parsed")
#                 collect()

#             except Exception as e:
#                 log_status_video(f"{video_id}_{thread_video_idx}",e)
#                 remove(output_alignment_path)
#                 remove(output_video_path)
                
#     except Exception as e:
#         log(f"Finished Clipping and filtering videos: {video_path}\n\n")
#         raise Exception(f"ERROR Threading problem: {e}")
    
#     log(f"Finished Clipping and filtering videos: {video_path}\n\n")
    
# def filter_video():
#     log("Filtering Videos\n\n")
#     try:
#         for i in glob(CLIPPED_VIDEOS+fr"/*.mp4"):
#             video_id = re.split(r"\\|/",i)[-1].split(".")[0]
#             video_path = i.replace("\\","/")
#             alignment_path = CLIPPED_ALIGNMENTS+"/"+video_id+".align"


#             try:
#                 video_is_accpeted(video_path)
#                 copy(video_path,FILTERED_VIDEOS)
#                 copy(alignment_path,FILTERED_ALIGNMENTS)
#                 log_status_video(video_id,"parsed")
#                 collect()
#             except Exception as e:
#                 log_status_video(video_id,e)
#         log("\nFinished Filtering Videos\n\n")
#     except Exception as e:
#         raise e

In [7]:
# def pipline(url, video_id,status,video_idx):
#     log(f"{'='*100}\n\n")
#     log(f"Start Pipline for id: {video_id} with status: {status}\n\n")
#     try:
#         if status == "parsed":
#             log(f"Finished Pipline for id: {video_id}\n\n")
#             log(f"{'='*100}\n\n")
#             return "parsed"
        
#         elif status == "downloaded":
#             empty_dir(CLIPPED_VIDEOS)
#             empty_dir(CLIPPED_ALIGNMENTS)

#             filter_video_threading(DOWNLOADED_VIDEOS+fr"/{video_id}.mp4", DOWNLOADED_ALIGNMENTS+fr"/{video_id}.srt",video_id)

#             empty_dir(CLIPPED_VIDEOS)
#             empty_dir(CLIPPED_ALIGNMENTS)
#             empty_dir(DOWNLOADED_VIDEOS)
#             empty_dir(DOWNLOADED_ALIGNMENTS)
#             log(f"Finished Pipline for id: {video_id}\n\n")
#             log(f"{'='*100}\n\n")
#             return "parsed"

#         # elif status == "clipped":
#         #     filter_video_threading()
#         #     empty_dir(CLIPPED_VIDEOS)
#         #     empty_dir(CLIPPED_ALIGNMENTS)
#         #     empty_dir(DOWNLOADED_VIDEOS)
#         #     empty_dir(DOWNLOADED_ALIGNMENTS)
#         #     log(f"Finished Pipline for id: {video_id}\n\n")
#         #     log(f"{'='*100}\n\n")
#         #     return "parsed"

            
#         elif status == "failed" or status == "unparsed":
#             empty_dir(CLIPPED_VIDEOS)
#             empty_dir(CLIPPED_ALIGNMENTS)
#             empty_dir(DOWNLOADED_VIDEOS)
#             empty_dir(DOWNLOADED_ALIGNMENTS)

#             download_video(url,video_id)
#             save_pipeline_status(video_idx,"downloaded")

#             filter_video_threading(DOWNLOADED_VIDEOS+fr"/{video_id}.mp4", DOWNLOADED_ALIGNMENTS+fr"/{video_id}.srt",video_id)
#             # save_pipeline_status(video_idx,"clipped")

#             empty_dir(CLIPPED_VIDEOS)
#             empty_dir(CLIPPED_ALIGNMENTS)
#             empty_dir(DOWNLOADED_VIDEOS)
#             empty_dir(DOWNLOADED_ALIGNMENTS)
#             log(f"Finished Pipline for id: {video_id}\n\n")
#             log(f"{'='*100}\n\n")
#             return "parsed"
        
#     except Exception as e:
#         log(f"ERROR Pipeline for {video_id}:\n\n")
#         log(f"Finished Pipline for id: {video_id}\n\n")
#         log(f"{'='*100}\n\n")
#         log(f"{e}\n\n")
#         return e    

In [8]:
df = read_csv(LINK_DATASET)
df.head()

,link,name,id,creator,status
0,https://www.youtube.com/watch?v=s0pbzSC81Kc,كتابة الكلام على الفيديو بشكل تلقائي بالذكاء ا...,0,0,clipped
1,https://www.youtube.com/watch?v=FYqNG3zwXL4&t=...,فيلم الصفاره,1,0,clipped
2,https://www.youtube.com/watch?v=3r3IOzNNKRM,جت كدا اياد الموجي وساره هاني,2,0,failed
3,https://www.youtube.com/watch?v=Z31F0zVuHOg&t=...,فيلم العارف,3,0,clipped
4,https://www.youtube.com/watch?v=0SGpG7m4xzE,فيلم الخليه,4,0,clipped


In [9]:
df[df['id'].isna()]        

,link,name,id,creator,status


In [10]:
df = df.dropna(subset=['id'])

In [11]:
# df['id'] = df['id'].astype(int).astype(str)


In [12]:
# df.to_csv(LINK_DATASET,index=False)

In [13]:
# df[df.duplicated('link',keep=False)]

In [14]:
# for idx,row in enumerate(df.values):
#     collect()
#     url = row[0]
#     video_id = row[2]
#     status = row[4]
#     # run pipline
#     result = pipline(url, video_id,status,idx)
#     # # save completion
#     save_pipeline_status(idx,result)
#     # df["status"][idx] = result
#     # df.to_csv(LINK_DATASET,index=False)    
#     print("Parsed Video : ",video_id)
#     collect()


In [15]:
# threads

In [16]:
# filter_video_threading(DOWNLOADED_VIDEOS+fr"/{video_id}.mp4", DOWNLOADED_ALIGNMENTS+fr"/{video_id}.srt",video_id)

In [17]:
# code in captions.py
    # def xml_caption_to_srt(self, xml_captions: str) -> str:
    #     """Convert xml caption tracks to "SubRip Subtitle (srt)".

    #     :param str xml_captions:
    #     XML formatted caption tracks.
    #     """
    #     segments = []
    #     root = ElementTree.fromstring(xml_captions)
    #     i=0
    #     for child in list(root.iter("body"))[0]:
    #         if child.tag == 'p':
    #             caption = ''
    #             if len(list(child))==0:
    #                 # instead of 'continue'
    #                 caption = child.text
    #             for s in list(child):
    #                 if s.tag == 's':
    #                     caption += ' ' + s.text
    #             caption = unescape(caption.replace("\n", " ").replace("  ", " "),)
    #             try:
    #                 duration = float(child.attrib["d"])/1000.0
    #             except KeyError:
    #                 duration = 0.0
    #             start = float(child.attrib["t"])/1000.0
    #             end = start + duration
    #             sequence_number = i + 1  # convert from 0-indexed to 1.
    #             line = "{seq}\n{start} --> {end}\n{text}\n".format(
    #                 seq=sequence_number,
    #                 start=self.float_to_srt_time_format(start),
    #                 end=self.float_to_srt_time_format(end),
    #                 text=caption,
    #             )
    #             segments.append(line)
    #             i += 1
    #     return "\n".join(segments).strip()

In [20]:
running_filtering= False

In [19]:
DOWNLOADER_THREADS = 0
CLIPPING_SLAVE_THREADS = 20
CLIPPING_THREADS = 10
FILTERING_THREADS = 0
downloader_threads = {}
clipper_threads ={}
filtering_threads = {}
running_clipping = True
running_downloading = False
running_filtering = False

In [20]:
def download_video(url,video_id,status):
    
    if status != "unparsed" :
        log(f"Skipping video: {video_id} with status {status}\n\n",LOG_FILE_DOWN)
        del downloader_threads[video_id]
        return
    
    if os.path.exists(fr"{DOWNLOADED_VIDEOS}/R_{video_id}.mp4"):
        log(f"Skipping video: {video_id} as it already exists with status {status}\n\n",LOG_FILE_DOWN)
        save_pipeline_status(video_id,"downloaded")
        del downloader_threads[video_id]
        return
        
    try:
        result = run([RUNTIME,DOWNLOADER,url,video_id],universal_newlines = True,stdout = PIPE,stderr = STDOUT)
    
        if "True" in result.stdout:
            rename(fr"{DOWNLOADED_VIDEOS}/{video_id}.mp4",fr"{DOWNLOADED_VIDEOS}/R_{video_id}.mp4")
            log(f"Downloaded: {video_id}\n\n",LOG_FILE_DOWN)
            save_pipeline_status(video_id,"downloaded")
            del downloader_threads[video_id]
            return
            
        else:
            if os.path.exists(f"{DOWNLOADED_VIDEOS}/{video_id}.mp4"):
                remove(f"{DOWNLOADED_VIDEOS}/{video_id}.mp4")
            
            log(f"ERROR: {video_id}\n\n",LOG_FILE_DOWN)
            save_pipeline_status(video_id,"failed")
            del downloader_threads[video_id]
            return
    
    except Exception as e:
        log(f"ERROR: {video_id} {str(e).strip()}\n\n",LOG_FILE_DOWN)
        save_pipeline_status(video_id,"failed")
        del downloader_threads[video_id]
        return 
    
def filter_slave(video_id,sub_id):

    try:
        
        result = run([RUNTIME,FILTERER ,video_id,sub_id],universal_newlines = True,stdout = PIPE,stderr = STDOUT)

        if "True" in result.stdout:
            rename(CLIPPED_VIDEOS+ f"/T_{video_id}_{sub_id}.mp4",CLIPPED_VIDEOS+ f"/P_{video_id}_{sub_id}.mp4")
            rename(FILTERED_VIDEOS+ f"/T_{video_id}_{sub_id}.mp4",FILTERED_VIDEOS+ f"/{video_id}_{sub_id}.mp4")
            log(f"Filtered video: {video_id}_{sub_id}\n\n",LOG_FILE_CLIP)

        else:
            rename(CLIPPED_VIDEOS+ f"/T_{video_id}_{sub_id}.mp4",CLIPPED_VIDEOS+ f"/X_{video_id}_{sub_id}.mp4")
            log(f"ERROR video: {video_id}_{sub_id}\n\n",LOG_FILE_CLIP)

        del filtering_threads[f"{video_id}_{sub_id}"]
        return
    
    except:
        del filtering_threads[f"{video_id}_{sub_id}"]
        return
     
def clip_slave(video_id, sub_id):
    try:

        if os.path.exists(CLIPPED_VIDEOS+ f"/R_{video_id}_{sub_id}.mp4") and os.path.exists(CLIPPED_ALIGNMENTS+f"/{video_id}_{sub_id}.align"):
            log(f"Skipping video: {video_id}_{sub_id}\n\n",LOG_FILE_CLIP)
            return
        
        result = run([RUNTIME,CLIPPER,video_id,sub_id],universal_newlines = True,stdout = PIPE,stderr = STDOUT)

        if "True" in result.stdout:
            rename(CLIPPED_VIDEOS+ f"/{video_id}_{sub_id}.mp4",CLIPPED_VIDEOS+ f"/R_{video_id}_{sub_id}.mp4")
            log(f"Clipped video: {video_id}_{sub_id}\n\n",LOG_FILE_CLIP)

        else:
            log(f"ERROR video: {video_id}_{sub_id}\n\n",LOG_FILE_CLIP)
        return
    except:
        return

def clip_manager(video_id,_):

    # clipper_slave_threads={}
    length = len(pysrt.open(fr"{DOWNLOADED_ALIGNMENTS}/{video_id}.srt"))
    for sub_id in range(length):
        if not running_clipping:
            del clipper_threads[video_id]
            return
        # while True:

        #     if len(clipper_slave_threads.keys()) < CLIPPING_SLAVE_THREADS:
        #         break
            
        #     for key, value in clipper_slave_threads.items():
        #         if time.time() - value[1] > 20:
        #             try:
        #                 log(f"ERROR video: {video_id}_{key}\n\n",LOG_FILE_CLIP)
        #                 value[0].join(timeout=1)
        #                 value[0].terminate()
        #                 del clipper_slave_threads[key]
        #             except:
        #                 pass
        clipper_threads[video_id][2] = round((sub_id / length) * 100,3)

        if (os.path.exists(CLIPPED_VIDEOS+ f"/P_{video_id}_{sub_id}.mp4") or os.path.exists(CLIPPED_VIDEOS+ f"/X_{video_id}_{sub_id}.mp4") or os.path.exists(CLIPPED_VIDEOS+ f"/R_{video_id}_{sub_id}.mp4")) and os.path.exists(CLIPPED_ALIGNMENTS+f"/{video_id}_{sub_id}.align"):
            log(f"Skipping video: {video_id}_{sub_id}\n\n",LOG_FILE_CLIP)
            continue
        clip_slave(str(video_id), str(sub_id))


        # thread = threading.Thread(target=clip_slave, args=(, str(sub_id), clipper_slave_threads),daemon=True)
        # clipper_slave_threads[sub_id] = [thread, time.time()]
        # thread.start()
    rename(DOWNLOADED_VIDEOS+f"/T_{video_id}.mp4",DOWNLOADED_VIDEOS+f"/P_{video_id}.mp4")
    save_pipeline_status(video_id,"clipped")
    del clipper_threads[video_id]
    return
        

In [21]:
def download_controller():
    
    for idx,row in enumerate(df.values):
        collect()
        if not running_downloading:
            break

        while True:
    
            if len(downloader_threads.keys()) < DOWNLOADER_THREADS:
                break

            # for key, value in dict(downloader_threads).items():
            #     if time.time() - value[1] > 800:
            #         try:
            #             save_pipeline_status(key,"inquiry")
            #             del downloader_threads[key]
            #         except:
            #             pass
                    
        url = str(row[0])
        video_id = str(row[2])
        status = str(row[4])

        thread = threading.Thread(target=download_video, args=(url,video_id,status),daemon= True)
        downloader_threads[video_id] = [thread,time.time()]
        thread.start()    
        
    
def clipping_controller():

    while running_clipping:
        for film in glob(DOWNLOADED_VIDEOS+"/R_*.mp4"):
            collect()
            if not running_clipping:
                break
            
            while True:
                if len(clipper_threads.keys()) < CLIPPING_THREADS:
                    break

            video_id = film.replace("\\","/").split("/")[-1].split(".mp")[0].removeprefix("R_").strip()
            rename(film,DOWNLOADED_VIDEOS+f"/T_{video_id}.mp4")
        

            thread = threading.Thread(target=clip_manager, args=(str(video_id),"_"),daemon=True)
            clipper_threads[video_id] = [thread,time.time(),0]
            thread.start()


def filtering_controller():

    while running_filtering:

        for clip in glob(CLIPPED_VIDEOS+"/R_*.mp4"):
            collect()
            if not running_filtering:
                break
            
            while True:
                if len(filtering_threads.keys()) < FILTERING_THREADS:
                    break

            video_id , sub_id= clip.replace("\\","/").split("/")[-1].split(".mp")[0].removeprefix("R_").strip().split("_")

            rename(clip,CLIPPED_VIDEOS+f"/T_{video_id}_{sub_id}.mp4")
        

            thread = threading.Thread(target=filter_slave, args=(str(video_id),str(sub_id)),daemon=True)
            filtering_threads[f"{video_id}_{sub_id}"] = [thread,time.time()]
            thread.start()  

In [22]:
import os

def rename_files(directory):
    # List all files in the directory
    files = os.listdir(directory)

    # Iterate through each file
    for filename in files:
        # Construct the full path of the file
        old_path = os.path.join(directory, filename)
        
        # Split the filename and extension
        base_name, extension = os.path.splitext(filename)
        new_base_name = ""
        # Check if the filename starts with "T_"
        if base_name.startswith("T_"):
            # Replace "T_" with "R_"
            new_base_name = base_name.replace("T_", "R_")
        # elif base_name.startswith("R_R_"):
        #     # Replace "T_" with "R_"
        #     new_base_name = base_name.replace("R_R_", "R_")
        # elif base_name.startswith("P_"):
        #     new_base_name = base_name.replace("P_", "R_")

        # elif base_name.startswith("X_"):
        #     new_base_name = base_name.replace("X_", "R_")

        # elif base_name.startswith("R_"):
        #     continue
        else:
            continue
        # else:
        #     os.remove(os.path.join(directory, base_name).replace("\\","/")+".mp4")      
        #     continue
        # Construct the new filename
        new_name = f"{new_base_name}{extension}"
        
        # Construct the full path of the new filename
        new_path = os.path.join(directory, new_name)
        
        os.rename(old_path, new_path)
        # Rename the file
        # if not os.path.exists(new_path):
        # else:
        #     os.remove(old_path)
        print(f"Renamed '{filename}' to '{new_name}'")

# Specify the directory containing the files
directory = DOWNLOADED_VIDEOS

# Call the function to rename files in the directory
rename_files(directory)

# # def empty_txt_files(dirs):
# #     for filename in dirs:
# #         try:
# #             open(filename, 'w').close()  # Open in write mode to truncate the file
# #             print(f"Emptied {filename}")
# #         except Exception as e:
# #             print(f"Error emptying {filename}: {e}")

# # # Example usage:
# # directories = [LOG_FILE_CLIP,LOG_FILE_DOWN,LOG_FILE_FILT]  # Add your directories here
# # empty_txt_files(directories)






Renamed 'T_175.mp4' to 'R_175.mp4'
Renamed 'T_187.mp4' to 'R_187.mp4'
Renamed 'T_188.mp4' to 'R_188.mp4'
Renamed 'T_189.mp4' to 'R_189.mp4'
Renamed 'T_190.mp4' to 'R_190.mp4'
Renamed 'T_196.mp4' to 'R_196.mp4'
Renamed 'T_197.mp4' to 'R_197.mp4'
Renamed 'T_199.mp4' to 'R_199.mp4'
Renamed 'T_200.mp4' to 'R_200.mp4'
Renamed 'T_201.mp4' to 'R_201.mp4'


In [15]:
downloader = threading.Thread(target=download_controller,daemon=True)

In [23]:
clipper = threading.Thread(target=clipping_controller,daemon=True)

In [74]:
filterer = threading.Thread(target=filtering_controller,daemon=True)

In [24]:
def time_format(seconds):
    hours = seconds // 3600
    remaining_seconds = seconds % 3600
    minutes = remaining_seconds // 60
    remaining_seconds %= 60
    
    return "{} hours, {} minutes, {} seconds".format(hours, minutes, remaining_seconds)


In [25]:
for process in [clipper]:
    process.start()

In [20]:
print("Current Downloader Threads: ")
print("Active threads: ",len(downloader_threads))
for key,value in downloader_threads.items():
    print(f"{key}: {time_format(int(time.time()-value[1]))}")


Current Downloader Threads: 
Active threads:  0


In [65]:
print("Current Clipper Threads: ")
print("Active threads: ",len(clipper_threads))
for key,value in clipper_threads.items():
    print(f"{key}: {value[2]}%    -- {time_format(int(time.time()-value[1]))}")



Current Clipper Threads: 
Active threads:  0


In [58]:
running_clipping = False

In [170]:
print("Current Filtering Threads: ")
print("Active threads: ",len(filtering_threads))
for key,value in filtering_threads.items():
    print(f"{key}: {time_format(int(time.time()-value[1]))}")


Current Filtering Threads: 
Active threads:  0


In [29]:
df = read_csv(LINK_DATASET)
print("Percentage of clipped videos: ",round((df['status'].value_counts()['clipped'] / (df['status'].value_counts()['clipped'] + df['status'].value_counts()['downloaded'])) * 100,4))
x = [ y for y in os.listdir(CLIPPED_VIDEOS)]
ready = [y for y in x if y.startswith("R")]
rejected_processed = [y for y in x if (y.startswith("X") or y.startswith("P"))]
print("Percentage of processed videos: ",round(len(rejected_processed) / (len(ready) + len(rejected_processed))*100,4))
del x
del ready
del rejected_processed
print("Amount of captured videos: ",len(os.listdir(FILTERED_VIDEOS)))


Percentage of clipped videos:  43.2773
Percentage of processed videos:  70.1245
Amount of captured videos:  4769


In [26]:
df = read_csv(LINK_DATASET)
df['status'].value_counts()

status
downloaded    120
clipped       118
failed         28
unparsed        1
Name: count, dtype: int64